In [ ]:
import plotly.express as px
from torchmetrics.regression import MeanAbsolutePercentageError, SymmetricMeanAbsolutePercentageError, MeanAbsoluteError, R2Score
from IPython.display import display, HTML

In [ ]:
def NormalizedRootMeanSquareError(target, preds):
    def RootMeanSquareError():
        return np.sqrt((1/target.shape[1]) * torch.pow((target-preds), 2).sum())
    
    RANGE = torch.max(target)-torch.min(target)

    return RootMeanSquareError() / RANGE

In [ ]:
def daily_mae(target:torch.Tensor, preds:torch.Tensor, days:int, day_in_timestamp: int):
    start = 0
    result = {}
    for day in range(days):
        t = pd.DataFrame(target.numpy().reshape(target.shape[1], 1))
        p = pd.DataFrame(preds.numpy().reshape(preds.shape[1], 1))
        
        t = t.iloc[start:start+day_in_timestamp]
        p = p.iloc[start:start+day_in_timestamp]
        
        start += day_in_timestamp
        
        # TODO: ricontrollare
        step_mae = float(abs(t.sum() - p.sum()))
        step_pmae = float(step_mae / t.sum()) * 100.0
        
        result[f'Day {day+1}'] = [step_mae, step_pmae]

    result = pd.DataFrame(result).T
    result.columns = ['MAE ()', "MAPE (%)"]
    
    # display(result)
    return result

In [ ]:
pd.set_option("display.precision", 2)

In [ ]:
def evaluate(model, finish_run=True):
    # model.load_state_dict(torch.load('GRRUN15MIN_b10_e100.model')) # TODO: rendere parametrico il path
    model.eval()
    test_dataset = CustomDataset(test_df, 1)
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=1,
        collate_fn=collate_fn,
        shuffle=False
    )

    result = pd.DataFrame()
    mean_abs_percentage_error = MeanAbsolutePercentageError()
    smape = SymmetricMeanAbsolutePercentageError()
    r2 = R2Score()
    mae = MeanAbsoluteError()
    nrmse = NormalizedRootMeanSquareError
    mape_points = []
    mape = 0

    # Create a table
    table = wandb.Table(columns = ["Evaluation (on Testing Set)", "MAPE(+)", "SMAPE", "NRMSE", "MAE", "R2"])

    # Create path for Plotly figure
    path_to_plotly_html = "./plotly_figure.html"

    for i, (vbefore, vtarget, vfuture, vafter, vm_b, vm_t, vm_a, timestamps, solargis) in enumerate(test_dataloader):
        timestamp =  pd.to_datetime(pd.Series(timestamps[0]))
        timestamp = timestamp[vm_t[0] == 1]

        vbefore, vafter, vfuture = vbefore.to(device), vafter.to(device), vfuture.to(device)
        prediction = model(vbefore, vafter, vfuture)
        prediction = prediction.cpu().detach()
        # prediction = prediction * vfuture.cpu().detach()[:,:,-1].reshape(prediction.shape) # output * isday
        prediction = (prediction/prediction.sum(dim=(1,2), keepdim=True)) * vtarget.sum(dim=(1,2), keepdim=True)

        # tutto quello che è < 0 diventa 0
        prediction[prediction < 0] = 0
        
        # TODO: ricontrollare mape
        #step_mape = mean_abs_percentage_error(target=vtarget, preds=prediction).item()
        step_mape_positives = mean_abs_percentage_error(target=vtarget[vtarget>0], preds=prediction[vtarget>0])
        step_smape = smape(target=vtarget, preds=prediction)
        step_mae = mae(target=vtarget, preds=prediction)
        step_r2 = r2(target=vtarget.squeeze(0), preds=prediction.squeeze(0))
        step_nrmse = nrmse(target=vtarget, preds=prediction)
        
        ndays = int(np.count_nonzero(vm_t[0]==1)/test_dataset.DAY_IN_TIMESTAMPS)
        
        step_dailymae = daily_mae(vtarget, prediction, ndays, test_dataset.DAY_IN_TIMESTAMPS)

        #vtarget = vtarget.numpy().reshape(vtarget.shape[1], 1)
        #prediction = prediction.numpy().reshape(prediction.shape[1],1)
        #step_mape_sklearn = mean_absolute_percentage_error(vtarget[vtarget>0], prediction[vtarget>0])

        mape_points.append(step_mape_positives)
        mape += step_mape_positives

        vtarget = pd.DataFrame(vtarget.numpy().reshape(vtarget.shape[1], 1))
        #vtarget = pd.DataFrame(vtarget)
        vtarget.columns = ['target']
        # target non normalizzata e cumulata
        unnormalized_target = pd.DataFrame((train_target_scaler.inverse_transform(vtarget)).cumsum())
        unnormalized_target.columns = ['target (real)']
        # target non normalizzata con differenze
        unnormalized_target_diff = pd.DataFrame(train_target_scaler.inverse_transform(vtarget))
        unnormalized_target_diff.columns = ['target (real)']

        prediction = pd.DataFrame(prediction.numpy().reshape(prediction.shape[1],1))
        #prediction = pd.DataFrame(prediction)
        prediction.columns = ['prediction']
        
        # preds non normalizzata e cumulata
        unnormalized_prediction = pd.DataFrame((train_target_scaler.inverse_transform(prediction)).cumsum())
        unnormalized_prediction.columns = ['prediction (real)']
        # preds non normalizzata con differenze
        unnormalized_prediction_diff = pd.DataFrame(train_target_scaler.inverse_transform(prediction))
        unnormalized_prediction_diff.columns = ['prediction (real)']
                                        
        solargis = solargis[0]
        
        step_result = pd.concat([vtarget, prediction], axis=1)
        step_result.index = timestamp
        # non normalizzata e cumsum
        step_result_real = pd.concat([unnormalized_target, unnormalized_prediction], axis=1)
        step_result_real.index = timestamp
        # non normalizzata con differenze
        step_result_real_diff = pd.concat([unnormalized_target_diff, unnormalized_prediction_diff], axis=1)
        step_result_real_diff.index = timestamp
        
        step_result = pd.concat([step_result, solargis], axis=1)
        
        fig = px.line(step_result, y=['target', 'prediction', 'GHI', 'GTI'], title=f"Step Prediction ({timestamp.dt.day.values[0]}-{timestamp.dt.month.values[0]}, {int(np.count_nonzero(vm_t[0]==1)/test_dataset.DAY_IN_TIMESTAMPS)} d)", markers=True)
        fig.write_html(path_to_plotly_html, auto_play=False) 
        fig.show()

        fig = px.line(step_result_real_diff, y=['target (real)', 'prediction (real)'], title=f"Step Real Diff Prediction ({timestamp.dt.day.values[0]}-{timestamp.dt.month.values[0]}, {int(np.count_nonzero(vm_t[0]==1)/test_dataset.DAY_IN_TIMESTAMPS)} d)", markers=True)
        fig.show()
                                                    
        fig = px.line(step_result_real, y=['target (real)', 'prediction (real)'], title=f"Step CumSum Prediction ({timestamp.dt.day.values[0]}-{timestamp.dt.month.values[0]}, {int(np.count_nonzero(vm_t[0]==1)/test_dataset.DAY_IN_TIMESTAMPS)} d)", markers=True)
        fig.show()

        table.add_data(wandb.Html(path_to_plotly_html), step_mape_positives, step_smape, step_nrmse, step_mae, step_r2)

        result =  pd.concat([result, step_result])
        
        step_realmae = mae(target=torch.tensor(step_result_real['target (real)'].values), preds=torch.tensor(step_result_real['prediction (real)'].values)).item()
        step_realmape = mean_abs_percentage_error(target=torch.tensor(step_result_real['target (real)'].values), preds=torch.tensor(step_result_real['prediction (real)'].values)).item()
        step_realsmape = smape(target=torch.tensor(step_result_real['target (real)'].values), preds=torch.tensor(step_result_real['prediction (real)'].values)).item()
        step_realr2 = r2(target=torch.tensor(step_result_real['target (real)'].values), preds=torch.tensor(step_result_real['prediction (real)'].values))

        step_real_dailymae = daily_mae(
            torch.tensor(step_result_real['target (real)'].to_numpy().reshape(1, step_result_real['target (real)'].shape[0], 1)), 
            torch.tensor(step_result_real['prediction (real)'].to_numpy().reshape(1, step_result_real['prediction (real)'].shape[0], 1)), ndays, test_dataset.DAY_IN_TIMESTAMPS)
        step_real_dailymae.columns = ['MAE (kWh)', 'MAPE (%)']
        
        
        step_realmae_diff = mae(target=torch.tensor(step_result_real_diff['target (real)'].values), preds=torch.tensor(step_result_real_diff['prediction (real)'].values)).item()
        step_realmape_diff = mean_abs_percentage_error(target=torch.tensor(step_result_real_diff['target (real)'].values), preds=torch.tensor(step_result_real_diff['prediction (real)'].values)).item()
        step_realsmape_diff = smape(target=torch.tensor(step_result_real_diff['target (real)'].values), preds=torch.tensor(step_result_real_diff['prediction (real)'].values)).item()
        step_realr2_diff = r2(target=torch.tensor(step_result_real_diff['target (real)'].values), preds=torch.tensor(step_result_real_diff['prediction (real)'].values))

        step_real_dailymae_diff = daily_mae(
            torch.tensor(step_result_real_diff['target (real)'].to_numpy().reshape(1, step_result_real_diff['target (real)'].shape[0], 1)), 
            torch.tensor(step_result_real_diff['prediction (real)'].to_numpy().reshape(1, step_result_real_diff['prediction (real)'].shape[0], 1)), ndays, test_dataset.DAY_IN_TIMESTAMPS)
        step_real_dailymae_diff.columns = ['MAE (kWh)', 'MAPE (%)']
        
        normdata_plot= pd.DataFrame(
            [step_mape_positives*100,step_smape*100,step_nrmse*100,step_mae,step_r2], columns=['Value'], index=['MAPE + (%)', 'SMAPE (%)', 'NRMSE (%)', 'MAE ()', 'R2'],
            dtype=float
        )
        normdata_plot = normdata_plot.astype(float).round(decimals=2)
        realdata_plot = pd.DataFrame(
            [step_realmae], index=['MAE (kWh)'], columns=['Value'],
            dtype=float
        )
        realdata_plot=realdata_plot.astype(float).round(decimals=2)
        
        realdata_diff_plot = pd.DataFrame(
            #[step_realmape_diff*100,step_realsmape_diff*100,step_realmae_diff,step_realr2_diff], columns=['Value'], index=['MAPE (%)', 'SMAPE (%)', 'MAE ()', 'R2'],
            [step_realmae_diff], columns=['Value'], index=['MAE (kW/h)'],
            dtype=float
        )
        realdata_diff_plot=realdata_diff_plot.astype(float).round(decimals=2)
        
        # plot Dataframes side by side
        html_str = (
            normdata_plot.style.set_table_attributes("style='display:inline; margin:3em;'").set_caption('<h3>Normalized Data Results</h3>').render() +
            step_dailymae.style.set_table_attributes("style='display:inline; margin:3em;'").set_caption('<h3>Normalized Daily MAE Results</h3>').render()+
            realdata_diff_plot.style.set_table_attributes("style='display:inline; margin:3em;'").set_caption('<h3>Real Data Results</h3>').render()+
            step_real_dailymae_diff.style.set_table_attributes("style='display:inline; margin:3em;'").set_caption('<h3>Real Daily MAE Results</h3>').render()

        )
        display(HTML(html_str))
    
    # Log Table
    run.log({"evluation_table": table})
    
    if finish_run:
        run.finish()